## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [29]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import numpy as np
import requests
import json
import time
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [37]:
url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
movies = []
imdb_ids = []

In [114]:
# main function, returns all relevant information we are looking for

def soupy(url, movies, imdb_ids):
    time.sleep(5)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    movies.extend(movie_deets(soup, movies))
    imdb_ids.extend(imdb_tags(soup, imdb_ids))
    print(len(imdb_ids))
    back = next_url(soup)
    if back:
        front = url.split('/chart')[0]
        new_url = front + back
        print(new_url)
        return soupy(new_url,movies, imdb_ids)
    else:
        return imdb_ids, movies

In [115]:
def imdb_tags(soup, imdb_ids):
    movie_info = soup.findAll('tr', style='')
    imdb_id = []
    for i in range(1,201):
        for detail in movie_info[i].find('td', class_='a-text-left mojo-field-type-title'):
            imdb_id.append(detail.get('href').split('/')[2]) 
    return imdb_id

In [116]:
# gets movie details in list format and returns to main function

def movie_deets(soup, movie_list):
    # navigates to where list of movies is
    movie_info = soup.findAll('tr', style='')
    movies = []
    
    # loops through all movies in the list and adds info to master movie list
    for i in range(1,201):
        movie_deets = []
        for detail in movie_info[i].findAll('td'):
            movie_deets.append(detail.text)
        movies.append(movie_deets)
    return movies

In [117]:
# gets the next url to navigate to the next page of results

def next_url(soup):
    
    #navigates to where url is
    next_ur = soup.find('li', class_='a-last').find('a') 
    
    #if url is there will return the next url
    # once the end of results is reached will return False to main function in order to terminate it
    if next_ur:
        return next_ur.get('href')
    else:
        return False

In [124]:
imdb = soupy(url, movies, imdb_ids)

200
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset=200
400
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400&area=XWW
600
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=600&area=XWW
800
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset=800
1000


In [36]:
len(imdb)

1000

In [125]:
df_mojo = pd.DataFrame(movies)

In [127]:
df_mojo.head()

,0,1,2,3,4,5,6,7
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018


In [128]:
def get_columns(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    headers = soup.find('div', class_='a-section imdb-scroll-table-inner')
    heads = headers.find('table').findAll('th')
    
    columns = []
    for head in heads:
        columns.append(head.text.split('\n')[0])
    return columns

In [129]:
df_mojo.columns = get_columns(url)

In [130]:
df_mojo.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018


In [131]:
df_mojo['imdb_id'] = imdb_ids

In [133]:
df_mojo.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year,imdb_id
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019,tt4154796
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009,tt0499549
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997,tt0120338
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015,tt2488496
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018,tt4154756


In [140]:
df_final = df_mojo.merge(df_tmdb, on='imdb_id', how='inner')

In [141]:
df_final.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year,imdb_id,adult,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019,tt4154796,False,...,2019-04-24,2797800564,181,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Part of the journey is the end.,Avengers: Endgame,False,8.3,13301
1,2,Avatar,"$2,790,439,000","$760,507,625",27.2%,"$2,029,931,375",72.8%,2009,tt0499549,False,...,2009-12-10,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.4,21338
2,3,Titanic,"$2,194,439,542","$659,363,944",30%,"$1,535,075,598",70%,1997,tt0120338,False,...,1997-11-18,1845034188,194,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.8,16981
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.3%,"$1,131,561,399",54.7%,2015,tt2488496,False,...,2015-12-15,2068223624,136,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.4,14462
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018,tt4154756,False,...,2018-04-25,2046239637,149,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An entire universe. Once and for all.,Avengers: Infinity War,False,8.3,18247


In [142]:
df_final.shape

(1000, 33)

In [143]:
import csv
df_final.to_csv('final_movie_data.csv')

In [3]:
api_key = '49fc59a029e4df5ad29e6be556dae829'

In [52]:
# 1 API using details method a list or For Loop
imdb_data = []
for ids in imdb:
    request = requests.get('https://api.themoviedb.org/3/movie/' + ids + '?api_key=' + api_key + '&language=en-US')
    i = request.json()
    imdb_data.append(i)
print(imdb_data)

[{'adult': False, 'backdrop_path': '/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg', 'belongs_to_collection': {'id': 86311, 'name': 'The Avengers Collection', 'poster_path': '/bvjltR2nDuhf1NlnwCrgNwvHuIh.jpg', 'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'}, 'budget': 356000000, 'genres': [{'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 28, 'name': 'Action'}], 'homepage': 'https://www.marvel.com/movies/avengers-endgame', 'id': 299534, 'imdb_id': 'tt4154796', 'original_language': 'en', 'original_title': 'Avengers: Endgame', 'overview': "After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.", 'popularity': 41.632, 'poster_path': '/or06FN3Dka5tukK1e9sl16pB3iy.jpg', 'production_companies': [{'id': 

In [53]:
request.json().keys()

dict_keys(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'])

In [54]:
for key in request.json().keys():
    print(key)
    value = request.json()[key]
    print(type(value))
    print('\n\n')

adult
<class 'bool'>



backdrop_path
<class 'str'>



belongs_to_collection
<class 'NoneType'>



budget
<class 'int'>



genres
<class 'list'>



homepage
<class 'str'>



id
<class 'int'>



imdb_id
<class 'str'>



original_language
<class 'str'>



original_title
<class 'str'>



overview
<class 'str'>



popularity
<class 'float'>



poster_path
<class 'str'>



production_companies
<class 'list'>



production_countries
<class 'list'>



release_date
<class 'str'>



revenue
<class 'int'>



runtime
<class 'int'>



spoken_languages
<class 'list'>



status
<class 'str'>



tagline
<class 'str'>



title
<class 'str'>



video
<class 'bool'>



vote_average
<class 'float'>



vote_count
<class 'int'>





In [136]:
df_tmdb = pd.DataFrame.from_dict(imdb_data)
df_tmdb.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,en,Avengers: Endgame,...,2019-04-24,2797800564,181,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Part of the journey is the end.,Avengers: Endgame,False,8.3,13301
1,False,/eS8rJ1KzRNBewx9MduiSHM4kr7S.jpg,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,...,2009-12-10,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.4,21338
2,False,/6VmFqApQRyZZzmiGOQq2C92jyvH.jpg,None,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,597,tt0120338,en,Titanic,...,1997-11-18,1845034188,194,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.8,16981
3,False,/k6EOrckWFuz7I4z4wiRwz8zsj4H.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,...,2015-12-15,2068223624,136,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.4,14462
4,False,/lmZFxXgJE3vgrciwuDib0N8CfQo.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/avengers-infinit...,299536,tt4154756,en,Avengers: Infinity War,...,2018-04-25,2046239637,149,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An entire universe. Once and for all.,Avengers: Infinity War,False,8.3,18247


In [45]:
df_tmdb.shape

(1000, 25)

In [66]:
df_tmdb.isnull().sum()

adult                      0
backdrop_path              0
belongs_to_collection    476
budget                     0
genres                     0
homepage                   0
id                         0
imdb_id                    0
original_language          0
original_title             0
overview                   0
popularity                 0
poster_path                0
production_companies       0
production_countries       0
release_date               0
revenue                    0
runtime                    0
spoken_languages           0
status                     0
tagline                    0
title                      0
video                      0
vote_average               0
vote_count                 0
dtype: int64

In [78]:
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
adult                   1000 non-null bool
budget                  1000 non-null int64
genres                  1000 non-null object
homepage                1000 non-null object
id                      1000 non-null int64
imdb_id                 1000 non-null object
original_language       1000 non-null object
original_title          1000 non-null object
overview                1000 non-null object
popularity              1000 non-null float64
production_companies    1000 non-null object
production_countries    1000 non-null object
release_date            1000 non-null object
revenue                 1000 non-null int64
runtime                 1000 non-null int64
spoken_languages        1000 non-null object
status                  1000 non-null object
tagline                 1000 non-null object
title                   1000 non-null object
video                   1000 non-null bool
v

In [67]:
# Drops entire columns
df_tmdb.drop(['backdrop_path','belongs_to_collection','poster_path'], axis = 1, inplace = True)

In [18]:
# Drops any row in subset that has NaN value
#df_tmdb.dropna(subset = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 
                         #'genres', 'homepage', 'id', 'imdb_id', 'original_language', 
                         #'original_title', 'overview', 'popularity', 'poster_path', 
                         #'production_companies', 'production_countries', 'release_date', 
                         #'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 
                         #'title', 'video', 'vote_average', 'vote_count'],inplace = True)

In [86]:
df_tmdb.head()

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,41.632,...,2019-04-24,2797800564,181,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Part of the journey is the end.,Avengers: Endgame,False,8.3,13301
1,False,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",35.640,...,2009-12-10,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.4,21338
2,False,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,597,tt0120338,en,Titanic,101-year-old Rose DeWitt Bukater tells the sto...,36.382,...,1997-11-18,1845034188,194,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.8,16981
3,False,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,38.945,...,2015-12-15,2068223624,136,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.4,14462
4,False,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/avengers-infinit...,299536,tt4154756,en,Avengers: Infinity War,As the Avengers and their allies have continue...,60.353,...,2018-04-25,2046239637,149,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An entire universe. Once and for all.,Avengers: Infinity War,False,8.3,18247


In [110]:
df_tmdb['genres'][0]

[{'id': 12, 'name': 'Adventure'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 28, 'name': 'Action'}]

In [104]:
df_tmdb['genres'][0][0]['name']

'Adventure'

In [92]:
df_tmdb.head()

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,main_genre
0,False,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,41.632,...,2797800564,181,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Part of the journey is the end.,Avengers: Endgame,False,8.3,13301,NaN
1,False,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",35.640,...,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.4,21338,NaN
2,False,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,597,tt0120338,en,Titanic,101-year-old Rose DeWitt Bukater tells the sto...,36.382,...,1845034188,194,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.8,16981,NaN
3,False,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,38.945,...,2068223624,136,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.4,14462,NaN
4,False,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/avengers-infinit...,299536,tt4154756,en,Avengers: Infinity War,As the Avengers and their allies have continue...,60.353,...,2046239637,149,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An entire universe. Once and for all.,Avengers: Infinity War,False,8.3,18247,NaN


In [128]:
# 2 API using Discover Method
request2 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' + api_key + '&language=en-US&sort_by=revenue.desc&page=1')
print(request2.text)
# json.loads(request2.text)

{"page":1,"total_results":10000,"total_pages":500,"results":[{"popularity":1.154,"vote_count":0,"video":false,"poster_path":"\/mtjOrFfukEBxokC184SaLwQklOA.jpg","id":707737,"adult":false,"backdrop_path":"\/mTD3TTznZqskwrqazMxXoO4XbWE.jpg","original_language":"en","original_title":"Borrasca","genre_ids":[27,9648],"title":"Borrasca","vote_average":0,"overview":"Sam Walker moves to the town of Drisking, Missouri where people inexplicably disappear.","release_date":"2018-05-29"},{"popularity":41.632,"id":299534,"video":false,"vote_count":13288,"vote_average":8.3,"title":"Avengers: Endgame","release_date":"2019-04-24","original_language":"en","original_title":"Avengers: Endgame","genre_ids":[12,878,28],"backdrop_path":"\/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg","adult":false,"overview":"After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo T

In [129]:
request2.json().keys()

dict_keys(['page', 'total_results', 'total_pages', 'results'])

In [130]:
request2.status_code

200

In [131]:
for key in request2.json().keys():
    print(key)
    value = request2.json()[key]
    print(type(value))
    print('\n\n')

page
<class 'int'>



total_results
<class 'int'>



total_pages
<class 'int'>



results
<class 'list'>





In [135]:
request2.json()['results'][1]

{'popularity': 41.632,
 'id': 299534,
 'video': False,
 'vote_count': 13288,
 'vote_average': 8.3,
 'title': 'Avengers: Endgame',
 'release_date': '2019-04-24',
 'original_language': 'en',
 'original_title': 'Avengers: Endgame',
 'genre_ids': [12, 878, 28],
 'backdrop_path': '/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg',
 'adult': False,
 'overview': "After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.",
 'poster_path': '/or06FN3Dka5tukK1e9sl16pB3iy.jpg'}

In [136]:
df_discover = pd.DataFrame.from_dict(request2.json()['results'])
df_discover

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,1.154,0,False,/mtjOrFfukEBxokC184SaLwQklOA.jpg,707737,False,/mTD3TTznZqskwrqazMxXoO4XbWE.jpg,en,Borrasca,"[27, 9648]",Borrasca,0.0,"Sam Walker moves to the town of Drisking, Miss...",2018-05-29
1,41.632,13288,False,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,299534,False,/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,en,Avengers: Endgame,"[12, 878, 28]",Avengers: Endgame,8.3,After the devastating events of Avengers: Infi...,2019-04-24
2,35.640,21327,False,/btnl50ZDJDSCal2NLQIYWw0XxvH.jpg,19995,False,/eS8rJ1KzRNBewx9MduiSHM4kr7S.jpg,en,Avatar,"[28, 12, 14, 878]",Avatar,7.4,"In the 22nd century, a paraplegic Marine is di...",2009-12-10
3,38.945,14455,False,/9rd002JS49RwDW944fF1wjU8iTV.jpg,140607,False,/k6EOrckWFuz7I4z4wiRwz8zsj4H.jpg,en,Star Wars: The Force Awakens,"[28, 12, 14, 878]",Star Wars: The Force Awakens,7.4,Thirty years after defeating the Galactic Empi...,2015-12-15
4,60.353,18240,False,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,299536,False,/lmZFxXgJE3vgrciwuDib0N8CfQo.jpg,en,Avengers: Infinity War,"[28, 12, 878]",Avengers: Infinity War,8.3,As the Avengers and their allies have continue...,2018-04-25
5,36.382,16972,False,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,597,False,/6VmFqApQRyZZzmiGOQq2C92jyvH.jpg,en,Titanic,"[18, 10749]",Titanic,7.8,101-year-old Rose DeWitt Bukater tells the sto...,1997-11-18
6,29.168,15586,False,/2c0ajTi8nvrsYl5Oi1lVi6F0kd2.jpg,135397,False,/xX0IzuFa1Fj06iU2NlOmeMPe7oS.jpg,en,Jurassic World,"[28, 12, 878, 53]",Jurassic World,6.6,Twenty-two years after the events of Jurassic ...,2015-06-06
7,59.109,5844,False,/2bXbqYdUdNVa8VIWXVfclP2ICtT.jpg,420818,False,/nRXO2SnOA75OsWhNhXstHB8ZmI3.jpg,en,The Lion King,"[12, 10751]",The Lion King,7.2,"Simba idolizes his father, King Mufasa, and ta...",2019-07-12
8,55.360,22417,False,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,24428,False,/kwUQFeFXOOpgloMgZaadhzkbTI4.jpg,en,The Avengers,"[28, 12, 878]",The Avengers,7.7,When an unexpected enemy emerges and threatens...,2012-04-25
9,27.624,7513,False,/d9jZ2bKZw3ptTuxAyVHA6olPAVs.jpg,168259,False,/7X6zOOEzXTQJkzDgBHVly1BgyYu.jpg,en,Furious 7,"[28, 53]",Furious 7,7.3,Deckard Shaw seeks revenge against Dominic Tor...,2015-04-01


In [101]:
df_discover.shape

(20, 14)

In [117]:
# 3 API using Find Method
imdb1_id = 'tt0878654'

request3 = requests.get('https://api.themoviedb.org/3/find/' + imdb1_id + '?api_key=' + api_key + '&language=en-US&external_source=imdb_id')
json.loads(request3.text)

{'movie_results': [{'adult': False,
   'backdrop_path': None,
   'genre_ids': [53],
   'id': 37503,
   'original_language': 'en',
   'original_title': 'Bulletface',
   'overview': 'Horror and Revenge drive this thriller set in the sex and human trafficking underbelly of a nightmarish city where spinal fluid has become the drug of choice...with frightening consequences.',
   'poster_path': None,
   'release_date': '2010-03-10',
   'title': 'Bulletface',
   'video': False,
   'vote_average': 5.5,
   'vote_count': 5,
   'popularity': 1.834}],
 'person_results': [],
 'tv_results': [],
 'tv_episode_results': [],
 'tv_season_results': []}

In [122]:
# 4 API using Search Method
search = 'Thor'

request4 = requests.get('https://api.themoviedb.org/3/search/movie?api_key=' + api_key + '&language=en-US&query=' + search + '&page=1')
json.loads(request4.text)


{'page': 1,
 'total_results': 168,
 'total_pages': 9,
 'results': [{'popularity': 32.38,
   'vote_count': 14866,
   'video': False,
   'poster_path': '/prSfAi1xGrhLQNxVSUFh61xQ4Qy.jpg',
   'id': 10195,
   'adult': False,
   'backdrop_path': '/c5tewtijj6aixi8zwwPuYJxifBA.jpg',
   'original_language': 'en',
   'original_title': 'Thor',
   'genre_ids': [28, 12, 14],
   'title': 'Thor',
   'vote_average': 6.7,
   'overview': "Against his father Odin's will, The Mighty Thor - a powerful but arrogant warrior god - recklessly reignites an ancient war. Thor is cast down to Earth and forced to live among humans as punishment. Once here, Thor learns what it takes to be a true hero when the most dangerous villain of his world sends the darkest forces of Asgard to invade Earth.",
   'release_date': '2011-04-21'},
  {'popularity': 42.502,
   'vote_count': 13873,
   'video': False,
   'poster_path': '/rzRwTcFvttcN1ZpX2xv4j3tSdJu.jpg',
   'id': 284053,
   'adult': False,
   'backdrop_path': '/kaIfm5r